In [87]:
import numpy as np
import open3d as o3d
import igl
import meshplot as mp

# Turn .PLY Mesh into .PLY2 Mesh
We need .PLY2 file to use Shin Yoshizawa (et al.)'s Crest Line code (http://www2.riken.jp/brict/Yoshizawa/Research/Crest.html)
<br>The .PLY2 file is made of 4 parts (http://www2.riken.jp/brict/Yoshizawa/Research/PLYformat/PLYformat.html)
<br>&emsp;this file format is good for: "approximation of the smooth surfaces" (quote from website)
<br>1. Line 1: # of vertices (Integer)
<br>2. Line 2: # of triangles (Integer)
<br>3. Set of vertex coordinates
<br>&emsp;1x vertex per line
<br>&emsp;Format: (Double x, Double y, Double z) <-Remove comma!
<br>4. Set of face information
<br>&emsp;face = triangle
<br>&emsp;Format: (3, Int x, Int y, Int z) <-Remove comma!
<br>&emsp;here, x, y, and z are vertex ID, which are integers

The PSEUDO_PLY is:
1st line is a number of vertices. 
2nd line is a number of triangles. 
3rd line is a neighborhood size (Integer) for fitting. 
4th line is 0 (no crest line tracing) or 1 (with crest line tracing). After these four lines, there are vertex coordinates and face information as PLY2 format but there is no 3 for face line (only three vertex IDs).

In [29]:
MESHFILE = "../Open3D_Test/LargeAreas/LargeArea_3/3_path=8.ply"
mesh = o3d.io.read_triangle_mesh(MESHFILE)
verts = np.asarray(mesh.vertices)
verts_count = len(verts)
print("Vertices: " + str(verts_count) + " Total")
print(verts)
faces = np.asarray(mesh.triangles)
faces_count = len(faces)
print("Faces(Triangles): " + str(faces_count) + " Total")
print(faces)

Vertices: 71060 Total
[[ -88.71260834 -139.15951538  -32.98026276]
 [ -91.72106934 -144.24497986  -32.98026276]
 [ -89.14536285 -139.76264954  -32.98026276]
 ...
 [-189.27557373  -93.81411743   -7.54122734]
 [-189.27392578  -92.96653748   -7.54092979]
 [-189.21330261  -92.32481384   -7.5490818 ]]
Faces(Triangles): 141827 Total
[[    1    15    14]
 [    1     3    15]
 [    3     7    15]
 ...
 [71059 70265 70301]
 [71059 70301 68275]
 [71059 68275 69732]]


In [37]:
print(verts[verts_count - 1])
print(faces[faces_count - 1])

[-189.21330261  -92.32481384   -7.5490818 ]
[71059 68275 69732]


In [38]:
PSEUDO_PLY2 = 'ply2.txt'
NEWLINE = '\n'
NEIGHBOR = 3
CRESTLINE = 0 # 0 means NO Crest line, 1 means YES Crest line
with open(PSEUDO_PLY2, 'w') as f:
    # num vertices
    f.write(str(verts_count) + NEWLINE)
    # num triangles
    f.write(str(faces_count) + NEWLINE)
    # neighborhood size
    f.write(str(NEIGHBOR) + NEWLINE)
    # Crestline Y/N
    f.write(str(1) + NEWLINE)
    # vertices (coordinates)
    for i in range(verts_count):
        f.write(" ".join(map(str, verts[i])) + NEWLINE)
    # faces (vertex IDs)
    for j in range(faces_count):
        f.write(" ".join(map(str, faces[j])) + NEWLINE)

1st line is a number of convex/concave crest line points (Integer). 
2nd line is a number of convex/concave crest line edges. 
3rd line is a number of crest line connected components (Integer).
Let us notate V, E, and N for above numbers, respectively. 
Starting from 4th line: 
there are V lines which include three Double and one Integer values in one line as "%lf %lf %lf %d": x,y,z of crest line point and the connected component ID. The line number of the file minus 4 represents a crest point ID. 
Next, there are N lines which include three Double values in one line: Ridgeness, Sphericalness and Cyclideness for each corresponded connected component ID (ex. 1st line of this section corresponds ID 0 of the connected component, 2nd line of them represents ID 1 and so on). 
Finally, there are E lines which include three Integer values: pair of crest points ID (edge) and the triangle ID of original mesh which includes this edge (if there is -1 of that triangle ID then that edge is a connecting edge)

In [62]:
f = open("test.txt", 'r')
line1 = int(f.readline())
print(line1)
line2 = int(f.readline())
print(line2)
line3 = int(f.readline())
print(line3)
vertices = np.zeros(shape = (3, 4))
for i in range(3):
    line = f.readline()
    vertices[i] = [float(n) for n in line.split()]
print(vertices)

111111
222222
333333
[[ 0.1   0.2   0.3   9.  ]
 [ 0.4   0.5   0.6  10.  ]
 [ 0.11  0.12  0.13 11.  ]]


In [85]:
def read_RnR(FILE):
    f = open(FILE, 'r')
    # num of vertices
    V = int(f.readline())
    # num of edges
    E = int(f.readline())
    # num of connected components
    N = int(f.readline())
    # vertices
    """[x, y, z, connected cmp ID]; index = vtx ID"""
    vertices = np.zeros(shape = (V, 4))
    for i in range(V): 
        #index = pointID
        line = f.readline()
        vertices[i] = [float(n) for n in line.split()]
    """print("first vtx")
    print(vertices[0])
    print("last vtx")
    print(vertices[V - 1])
    print(vertices.dtype)"""
    # connected components
    """[Ridgeness, Sphericalness, Cyclideness]; index = connected cmp ID"""
    connected_cmp = np.zeros(shape = (N, 3))
    for j in range(N):
        line = f.readline()
        connected_cmp[j] = [float(n) for n in line.split()]
    """print("first connected cmp")
    print(connected_cmp[0])
    print("last connected cmp")
    print(connected_cmp[N - 1])
    print(connected_cmp.dtype)"""
    # edges (u,v)
    """[vtx ID of u, vtx ID of v, triangle ID]; index = edge ID"""
    edges = np.zeros(shape = (E, 3), dtype=int)
    for k in range(E):
        line = f.readline()
        edges[k] = [n for n in line.split()]
    """print("first edge")
    print(edges[0])
    print("last edge:")
    print(edges[E - 1])
    print(edges.dtype)"""
    return V, E, N, vertices, edges, connected_cmp

In [ ]:
RIDGES = "ridges.txt"
RI_V, RI_E, RI_N, RI_vtx, RI_edg, RI_cmp = read_RnR(RIDGES)
RAVINES = "ravines.txt"
RA_V, RA_E, RA_N, RA_vtx, RA_edg, RA_cmp = read_RnR(RAVINES)

In [114]:
RI_vtx[RI_edg[i, 0]][0:3]


array([ -85.755051, -127.727608,  -29.660699])

In [ ]:
def show_RnR(FILE):
# reading from file
    f = open(FILE, 'r')
    # num of vertices
    V = int(f.readline())
    # num of edges
    E = int(f.readline())
    # num of connected components
    N = int(f.readline())
    # vertices
    """[x, y, z, connected cmp ID]; index = vtx ID"""
    vertices = np.zeros(shape = (V, 4))
    for i in range(V): 
        #index = pointID
        line = f.readline()
        vertices[i] = [float(n) for n in line.split()]
    # connected components
    """[Ridgeness, Sphericalness, Cyclideness]; index = connected cmp ID"""
    connected_cmp = np.zeros(shape = (N, 3))
    for j in range(N):
        line = f.readline()
        connected_cmp[j] = [float(n) for n in line.split()]
    # edges (u,v)
    """[vtx ID of u, vtx ID of v, triangle ID]; index = edge ID"""
    edges = np.zeros(shape = (E, 3), dtype=int)
    for k in range(E):
        line = f.readline()
        edges[k] = [n for n in line.split()]
# plot base mesh
p = mp.plot(verts, faces, return_plot=True)


In [117]:
p = mp.plot(verts, faces, return_plot=True)
RI_s = np.zeros(shape = (RI_E, 3))
RI_t = np.zeros(shape = (RI_E, 3))
for i in range(RI_E):
    RI_s[i] = RI_vtx[RI_edg[i, 0]][0:3]
    RI_t[i] = RI_vtx[RI_edg[i, 1]][0:3]
p.add_lines(RI_s, RI_t, shading={"line_color": "red", "line_width": 10.0})
# p.add_lines(verts[RA_edg[:,0]], verts[RA_edg[:,1]], shading={"line_color": "blue", "line_width": 10.0})

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(-140.0000…

1

In [42]:
V = 4
vertices = np.zeros(shape = (3, 2))
print(vertices)

[[0. 0.]
 [0. 0.]
 [0. 0.]]


In [69]:
vertices = np.array([
    [1,2],
    [3,4]
    ])
print(vertices.shape)
print(vertices[1])
print(vertices[0, 1])
print(vertices)

(2, 2)
[3 4]
2
[[1 2]
 [3 4]]
